#### About the Notebook
This notebook is to showcase how you can use PyTorch for Logistic Regression. This notebook is inspired by the series "PyTorch: Zero to GANs" by Aakash NS.

#### Import all the modules

In [1]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)
import numpy as np

1.0.1.post2
0.2.1


#### Download the dataset from torchvision

In [2]:
from torchvision.datasets import MNIST

In [3]:
### MNIST has two sections - Train and Test
### To download the train section we will pass the directory where data is to be downloaded, specify train=True
### so that only train section is chosen and then specify download=True so that the download happens
train = MNIST(root='data/',train=True,download=True)

In [4]:
### Even if there is no directory called data, it will be created and train section will be downloaded
### We will follow similar practice to download the test section. The only change will be that 
### we will specify train=False so that the test section is downloaded
test = MNIST(root='data/',train=False,download=True)

The data is now downloaded. Let us now check the size of the downloaded train and test section of the dataset

In [5]:
len(train), len(test)

(60000, 10000)

In [6]:
### Let us look into what the train section contains
train[0]

(<PIL.Image.Image image mode=L size=28x28 at 0x11A274198>, tensor(5))

In [7]:
### We can see above that the first element in the train section contains an PIL Image in grayscale (mode=L)
### and a label of type tensor. 
### Lets get both the values from the first element
img,label = train[0]

In [8]:
### Let's see the image
img

In [9]:
### Let's see the label. Since the label is a single tensor element, we will pass label.item() to see the value only
label.item()

5

As you can see above the image is still in picture format. It has to be transformed to a Tensor if it has to be processed by PyTorch. Hence we will be doing the transformation to convert the same into a Tensor

In [10]:
tfms = torchvision.transforms.ToTensor()

In [11]:
train.transform = tfms

In [12]:
img,label = train[0]
img
print(img.shape)

torch.Size([1, 28, 28])


In [13]:
### We will apply the same to test and see the results
test.transform = tfms

In [14]:
train[0][0].shape

torch.Size([1, 28, 28])

#### Split the Train Dataset into Train and Validation Sections

The downloaded dataset is already a PyTorch Dataset type. So we can directly go and create the DataLoader

In [15]:
type(train)

torchvision.datasets.mnist.MNIST

Before creating the dataloaders, let's find a way to split the train dataset into a training and validation set.

One way is to use torch.utils.data.random_split and specify the split sizes on the dataset length. In the case below, I have specified out of total 60000 items in the dataset, 54000 items will be in train and 6000 will be in validation.

In [16]:
train_ds,val_ds = torch.utils.data.random_split(train,[int(len(train)*0.9),int(len(train)*0.1)])

As you can see the split has been performed well and we are able to retrieve the items in the split datasets below.

In [17]:
train_ds[0],val_ds[0]

((tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
            0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
            0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
            0.0000, 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
            0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
            0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
            0.0000, 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
            0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
            0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
            0.0000, 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
            0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
            0.0000, 0.0000, 0

Another way is to use the SubsetRandomSampler from PyTorch. This is in the torch.utils.data.sampler section and takes in indices as values. It samples batch size of indices from the total indices provided without replacement. 

So, we will split the 60000 indices available in the train dataset randomly into two sets that contain 54000 and 6000. We will then provide these randomly split indices to the SubsetRandomSampler.

In [18]:
### We will randomly rearrange the 60000 indices using np.random.permutation
### In the train dataset the indices are numbered in an ordered manner from 0 to 59999
### We will generate a random order of indices which total 60000 in number but do not contain the orderliness
### that was there before
idx = np.random.permutation(len(train))
idx

array([ 1232, 11667,  4020, ..., 34816, 26699, 56766])

In [19]:
### As you can see above the orderliness has now been disturbed
train_indices = idx[:int(len(train)*0.9)]
val_indices = idx[int(len(train)*0.9):]
len(train_indices),len(val_indices)

(54000, 6000)

The Dataloaders can now be defined.

In [20]:
### Please note that if you are using the sampler then Shuffle has to be False
### Hence we are not shuffling the train or validation dataloader
train_dl = torch.utils.data.DataLoader(train,batch_size=32,shuffle=False,
                                       sampler=torch.utils.data.sampler.SubsetRandomSampler(train_indices))
val_dl = torch.utils.data.DataLoader(train,batch_size=32,shuffle=False,
                                       sampler=torch.utils.data.sampler.SubsetRandomSampler(val_indices))

In [21]:
len(train)

60000

#### Create the Model

In [22]:
import torch.nn as nn

In [23]:
import torch.nn.functional as F

We are creating a very simple Model which is nothing but a Linear Layer. Loss function is cross entropy and Optimizer used is SGD with learning rate of 1e-3

In [24]:
model = nn.Linear(784,10)
loss_fn = F.cross_entropy
opt = torch.optim.SGD(model.parameters(),lr=1e-3)

In [26]:
## Total number of batches in Train and Validation Dataloaders
print("Total number of batches in Train Data Loader is {}".format(len(train_dl)))
print("Total number of batches in Validation Data Loader is {}".format(len(train_dl)))

Total number of batches in Train Data Loader is 1688
Total number of batches in Validation Data Loader is 1688


#### Define Helper Functions for getting Accuracy per Batch, Loss Per Batch, Accuracy per Epoch and Loss Per Epoch

In [27]:
## We take the output of the model, targets and batch_size
## We take the softmax of the output to get the probabilities
## The maximum of the probabilities is taken as the predicted label value
## Accuracy is defined as number of predictions that equal the target values divided by the batch size
def accuracy_batch(output,yb,batch_num):
    probs = F.softmax(output,dim=1)
    _,preds = torch.max(probs,dim=1)
    acc_batch = torch.sum(preds==yb).item()/batch_num
    return acc_batch

In [28]:
## We take the loss function, input and target values
## Input is resized from [1,28,28] to [batch_size,784] so as to flatten the values for processing
## Loss is calculated by passing the loss function
## Accuracy of the batch is calculated by calling the accuracy_batch function
## Loss, Value of Loss, Accuracy of batch and batch size is returned
def loss_batch(loss_fn,xb, yb):
    output = model(xb.reshape(-1,784))
    loss_bat = loss_fn(output,yb)
    batch_num = len(xb)
    acc_batch = accuracy_batch(output,yb,batch_num)
    return loss_bat,loss_bat.item(),acc_batch,batch_num

In [29]:
## Total Accuracy per Epoch is calculated by taking the accuracy per batch and 
## dividing by total number of batches
def avg_accuracy(acc_batch,batch_num,total_batches):
    avg_acc = np.sum(acc_batch)/total_batches
    return avg_acc

In [30]:
## Total Loss per Epoch is calculated by taking the loss per batch and 
## dividing by total number of batches
def avg_loss(loss_batch,batch_num,total_batches):
    return np.sum(loss_batch)/total_batches

In [31]:
## We define a function that takes the number of epochs for training,model, 
## the dataloaders (train and validation) and the optimizer function
def fit(epochs,model,loss_fn,train_dl,val_dl,opt):
    ## Total number of batches for train and validation are found
    num_bat_train = len(train_dl)
    num_bat_val = len(val_dl)
    ## Iterate training through the number of epochs
    for i in range(epochs):
        ## Capture the per batch accuracy, batch size, batch loss
        avg_bat_loss = []
        avg_bat_acc = []
        avg_batch_n = []
        ## Iterate through a complete epoch
        for xb,yb in train_dl:
            ## Get the loss per batch, loss value, accuracy per batch and batch size
            loss,loss_bat,acc_batch,batch_num = loss_batch(loss_fn,xb,yb)
            ## Append the above got values into a list defined at the start
            avg_bat_loss.append(loss_bat)
            avg_bat_acc.append(acc_batch)
            avg_batch_n.append(batch_num)
            ## Perform Gradient Descent and Optimization
            loss.backward()
            opt.step()
            opt.zero_grad()
        ## Calculate the Accuracy and Loss per Epoch and print them
        epoch_loss = avg_loss(avg_bat_loss,avg_batch_n,num_bat_train)
        epoch_accuracy = avg_accuracy(avg_bat_acc,avg_batch_n,num_bat_train)
        print("Training Epoch {}/{}, Loss is {:.4f}, Accuracy is {:.4f}".format(i+1,epochs,epoch_loss,epoch_accuracy))
        ## Perform the same process for validation dataset using validation dataloader
        ## But we ask for the gradients not to be calculated and updated
        val_loss = []
        val_acc = []
        batch_v = []
        for img_v,label_v in val_dl:
            with torch.no_grad():
                img_v = img_v.reshape(-1,784)
                loss_v,loss_vt,acc_bat_v,batch_num_v = loss_batch(loss_fn,img_v,label_v)
                val_loss.append(loss_vt)
                batch_v.append(batch_num_v)
                val_acc.append(acc_bat_v)
        avg_val_loss = avg_loss(val_loss,batch_v,num_bat_val)
        avg_val_accuracy = avg_accuracy(val_acc,batch_v,num_bat_val)
        print("Validation Epoch {}/{}, Loss is {:.4f}, Accuracy is {:.4f}".format(i+1,epochs,
                                                                                  avg_val_loss,avg_val_accuracy))

In [32]:
fit(5,model,loss_fn,train_dl,val_dl,opt)

Training Epoch 1/5, Loss is 1.6832, Accuracy is 0.6636
Validation Epoch 1/5, Loss is 1.2801, Accuracy is 0.7960
Training Epoch 2/5, Loss is 1.0885, Accuracy is 0.8108
Validation Epoch 2/5, Loss is 0.9535, Accuracy is 0.8261
Training Epoch 3/5, Loss is 0.8637, Accuracy is 0.8329
Validation Epoch 3/5, Loss is 0.8042, Accuracy is 0.8384
Training Epoch 4/5, Loss is 0.7478, Accuracy is 0.8454
Validation Epoch 4/5, Loss is 0.7163, Accuracy is 0.8454
Training Epoch 5/5, Loss is 0.6761, Accuracy is 0.8530
Validation Epoch 5/5, Loss is 0.6604, Accuracy is 0.8492


In [33]:
fit(5,model,loss_fn,train_dl,val_dl,torch.optim.SGD(model.parameters(),lr=1e-3))

Training Epoch 1/5, Loss is 0.6268, Accuracy is 0.8584
Validation Epoch 1/5, Loss is 0.6199, Accuracy is 0.8549
Training Epoch 2/5, Loss is 0.5905, Accuracy is 0.8637
Validation Epoch 2/5, Loss is 0.5898, Accuracy is 0.8549
Training Epoch 3/5, Loss is 0.5625, Accuracy is 0.8671
Validation Epoch 3/5, Loss is 0.5660, Accuracy is 0.8600
Training Epoch 4/5, Loss is 0.5401, Accuracy is 0.8700
Validation Epoch 4/5, Loss is 0.5463, Accuracy is 0.8627
Training Epoch 5/5, Loss is 0.5217, Accuracy is 0.8727
Validation Epoch 5/5, Loss is 0.5302, Accuracy is 0.8645


In [34]:
## Save the Model state containing the weights and other information
torch.save(model.state_dict(),'mnist.pth')

In [35]:
## Get a single item from the test dataset
img,label = test[0]
img.shape

torch.Size([1, 28, 28])

In [36]:
## Add dimension to the image to add the batch size dimension
## As seen above it is three dimensional shape but since the model performs on a batch
## it should be converted to have another dimension that signifies the batch size
img = img.unsqueeze(0)

In [37]:
## Predict the label and check with actual for the image
## As is norm we will need to convert the input into a two dimensional shape
preds = model(img.reshape(-1,784))
probs = F.softmax(preds,dim=1)
_,pred_label = torch.max(probs,dim=1)
print(pred_label,label)

tensor([7]) tensor(7)
